## CORDIC TESTBENCH


This notebook is to test the implementation of Cordic . It converts cartesian to polar coordinates. The output is compared with a python implementation of the same. It takes in x and y and gives out r and theta where r is the radius and theta is the angle.

In [1]:
from pynq import Overlay
from pynq import MMIO
import numpy as np
import struct
import binascii
import cmath
import random

In [2]:
ol=Overlay('./cordic.bit') #Change name of bitstream as required

In [3]:
cordic_ip=MMIO(0x43C00000,10000) #Change base address as required

In [4]:
ct=0
for i in range(10):
    #Generating random inputs
    x=random.uniform(-1,1)
    y=random.uniform(-1,1)
    
    #Computing golden output
    cn=complex(x,y)
    cn=cmath.polar(cn)
    
    #Converting input to bytes to be sent to FPGA
    x=(struct.unpack('<I', struct.pack('<f', x))[0])
    y=(struct.unpack('<I', struct.pack('<f', y))[0])
    
    #Writing values to the FPGA
    cordic_ip.write(0x10,x)                             #Change the offset as mentioned in vivado file
    cordic_ip.write(0x18,y)                             #Change the offset as mentioned in vivado file
    
    #Starting and stopping the IP (Don't change this)
    cordic_ip.write(0x00,1)
    cordic_ip.write(0x00,0)
    
    #Reading from IP
    r=hex(cordic_ip.read(0x20))                         #Change the offset as mentioned in vivado file
    r=r[2:]
    theta=hex(cordic_ip.read(0x28))                     #Change the offset as mentioned in vivado file
    theta=theta[2:]
    
    #Converting to float
    if r!=0:
        r=struct.unpack('>f', binascii.unhexlify(r))
        r=r[0]
    if theta!=0:
        theta=struct.unpack('>f', binascii.unhexlify(theta))
        theta=theta[0]
        
    #Printing and Comparing with golden output    
    print("r:",r,"Golden r:",cn[0],"theta:",theta,"Golden theta:",cn[1])
    if (abs(r-cn[0])<0.001) and (abs(theta-cn[1])<0.001):
        ct=ct+1

if ct==10:
    print("PASSED")
else:
    print("FAILED")

r: 0.6484100818634033 Golden r: 0.6484665141905163 theta: -0.590743362903595 Golden theta: -0.5907167493612641
r: 0.4880574345588684 Golden r: 0.4881000973246872 theta: 1.4922879934310913 Golden theta: 1.4923124315191336
r: 1.3128074407577515 Golden r: 1.3129217835139513 theta: -0.8384329080581665 Golden theta: -0.8384381745054351
r: 0.9772644639015198 Golden r: 0.977349585891099 theta: 3.037658452987671 Golden theta: 3.0377840523086075
r: 0.5137990713119507 Golden r: 0.5138438980900786 theta: -0.6283385157585144 Golden theta: -0.6283393992567109
r: 0.8309587836265564 Golden r: 0.8310311938119501 theta: -0.941987156867981 Golden theta: -0.9420033874791789
r: 0.4481015205383301 Golden r: 0.448140611301344 theta: 0.6595706343650818 Golden theta: 0.6595778281702334
r: 0.33386513590812683 Golden r: 0.3338942523951831 theta: 1.5257877111434937 Golden theta: 1.5257626636516133
r: 0.842417299747467 Golden r: 0.8424907450548891 theta: 1.7376888990402222 Golden theta: 1.7377549398757908
r: 0.67